In [1]:
import numpy as np
import random
import timeit
import math
from scipy import stats
DEBUG = 0

Parte 1

Executar uma simulação de horizonte infinito para estimar o tempo médio de espera em uma fila M/M/1.

Eliminar o transiente usando a heurística MSER -5Y.

Usar o método Batch Means para estimar o tempo médio de espera na fila e o respectivo intervalo de confiança de 95%.

A regra de parada é a precisão relativa de 5%.

Cenário I: λ = 7 clientes por segundo; μ =10 clientes por segundo.

Cenário II: λ = 8 clientes por segundo; μ =10 clientes por segundo.

Cenário III: λ = 9 clientes por segundo; μ =10 clientes por segundo.

Cenário IV: λ = 9,5 clientes por segundo; μ =10 clientes por segundo.

Plotar um gráfico comparativo dos quatro cenários.

Plotar uma linha horizontal para o valor teórico esperado em cada cenário.

Parte 2

Executar uma simulação de horizonte infinito para estimar o tempo médio de espera em uma fila M/M/1.

Eliminar o transiente usando a heurística MSER -5Y.

Usar o método Overpapping Batch Means para estimar o tempo médio de espera na fila e o respectivo intervalo de confiança de 95%.

A regra de parada é a precisão relativa de 5%.

Cenário I: λ = 7 clientes por segundo; μ =10 clientes por segundo.

Cenário II: λ = 8 clientes por segundo; μ =10 clientes por segundo.

Cenário III: λ = 9 clientes por segundo; μ =10 clientes por segundo.

Cenário IV: λ = 9,5 clientes por segundo; μ =10 clientes por segundo.

Plotar um gráfico comparativo dos quatro cenários.

Plotar uma linha horizontal para o valor teórico esperado em cada cenário.


Parte 3

Executar uma simulação de horizonte infinito para estimar o tempo médio de espera em uma fila M/M/1.

Eliminar o transiente usando a heurística MSER -5Y.

Usar o método Overpapping Batch Means para estimar o tempo médio de espera na fila e o respectivo intervalo de confiança de 95%.

A regra de parada é a precisão relativa de 5%.

Cenário I: λ = 7 clientes por segundo; μ =10 clientes por segundo.

Cenário II: λ = 8 clientes por segundo; μ =10 clientes por segundo.

Cenário III: λ = 9 clientes por segundo; μ =10 clientes por segundo.

Cenário IV: λ = 9,5 clientes por segundo; μ =10 clientes por segundo.

Plotar um gráfico comparativo dos quatro cenários.

Plotar uma linha horizontal para o valor teórico esperado em cada cenário.

Repetir para diferentes graus de superposição (overlap): 10%, 25% e 50%.

### Criação da lista de variáveis aleatórias

In [2]:
# Gerará uma lista de variáveis aleatórias a partir da função RANDOM
def G5(n):
    random.seed(0)

    vet_rn = []
    for i in range(0,n,1):
        vet_rn.append(random.random())
    
    return(vet_rn)

In [3]:
# Transforma uma lista de variáveis aleatórias em uma distribuição aleatória uniforme entre [0,1]
def random_list(vet,B):
    Ts = np.zeros(len(vet))
    for i in range(0,len(vet)):
        Ts[i] = -B*np.log(1-vet[i])
    return list(Ts)

### Função para formação do intervalo de entrada

In [4]:
# Soma-se o próximo com o anterior para ter uma noção de tempo de chagada. Sempre o tempo de chegada do
#primeiro será 0.
def sum_intervals(vet):
    for i in range(1, len(vet)):
        vet[i] += vet[i-1]
    return vet

### Função para calcular intervalo de confiança

In [5]:
# Dado uma lista de valores de processamento MM1, calcula-se com a função stats.t.interval o intervalo
#de confiança para 95%
def calculate_confidence_95(mm1):
    return (stats.t.interval(confidence=0.95, df = len(mm1)-1, loc=np.mean(mm1), scale=stats.sem(mm1)))

### MSER-5Y

In [6]:
def ZMean(ZVet, k, d):
    mean = 0
    for i in range(d, k):
        mean += ZVet[i]
        
    mean = mean/(k-d)
    return mean
    
def ZDP(ZVet, k, d, ZVetMean):
    dp = 0
    for i in range(d, k):
        dp += (ZVet[i] - ZVetMean)**2

    dp = dp/(k-d)
    
    return dp
    
def MSER5(k, d, ZVetVar):
    return ZVetVar/(k-d)**(1/2)

def MSER5Y(vet, d):
    N = len(vet)
    m = 5
    k = N/m
    if(DEBUG == 1): 
        print("N = ", len(vet))
        print("m = 5")
        print("k = ", k, "\n")
    
    k = int(k)
    ZVet = np.zeros(k)
    #if(DEBUG == 1): print("Blocos de cálculo do Z")
    for j in range(1,k+1):
        for i in range(1,m+1):
            index = (m*(j-1)+i)-1
            #if(DEBUG == 1):print("index: ",m,"* (",j-1,") +",i, "=", index)
            ZVet[j-1] += vet[index]
        
        #if(DEBUG == 1): print("\n")

    ZVet = ZVet/m
    #if(DEBUG == 1): print("Vetor Z: ", ZVet)
    
    ZVetMean = ZMean(ZVet, k, d)
    if(DEBUG == 1): print("Z Média = ",ZVetMean)
    
    ZVetDP = ZDP(ZVet, k, d, ZVetMean)
    ZVetVar = ZVetDP**(1/2)
    if(DEBUG == 1): print("Z Desvio Padrão = ",ZVetDP)
    if(DEBUG == 1): print("Z Variância = ",ZVetVar)
    
    mser5 = MSER5(k, d, ZVetVar)
    if(DEBUG == 1): print("\nMSER5 = ",mser5)
    
    return mser5

def MSER5YLoop(vet):
    MSER5YVet = []
    meiok = (len(vet)/5)/2
    meiok = int(math.ceil(meiok))
    
    for i in range(0, meiok):
        MSER5YVet.append(MSER5Y(vet, i))
    
    minimo = min(MSER5YVet)
    indexTransiente = minimo*5
    indexTransiente = int(math.ceil(indexTransiente))
    qtdMinimo = MSER5YVet.count(minimo)
    
    if(indexTransiente > len(vet)/2 or qtdMinimo > 1):
        print("Pontos de Truncagem ainda não encontrados")
        return 0
    else: 
        #print("Transiente:", indexTransiente)
        return indexTransiente

### Fila MM1

In [7]:
def process_queue(tx_service, tx_entry, is_run):
    # Wait = Tempo de espera total
    # tx_service = Lista com taxas de serviço
    # tx_entry = Lista com taxas de entrada
    # is_run = Flag que serve para quebrar o loop infinito de chamada dessa função
    clock = 0
    wait = 0
    wait_list = []
    bff = 0
    for i in range(0,len(tx_service)):
        wait = clock - tx_entry[i]
        wait_list.append(wait)
        clock += tx_service[i]
    
    confidence = calculate_confidence_95(wait_list)
    
    Table_value=1074.679
    if(Table_value >= confidence[0] and Table_value <= confidence[1]): is_run = False
    
    indexTransiente = MSER5YLoop(wait_list)
    
    return is_run, indexTransiente 

### Execução

In [8]:
def start_cicle(ElementsToAdd, H, U):
    #ElementsToAdd = Elementos adicionados por ciclo
    #U(μ) = taxa de entrada
    #H(λ) = taxa de serviço

    U = 1/U
    H = 1/H
    
    clock = 0
    wait = 0
    tx_service = []
    tx_entry = []
    wait_list = []
    
    is_run = True #Flag para quebrar a geração infinita
    first_run = True
    inicio = timeit.default_timer()
    
    while(is_run):
        random_arr = G5(ElementsToAdd)
        tx_service = tx_service + random_list(random_arr,U)
        
        bff = random_list(random_arr,H)
        
        if(first_run == False):
            bff = [tx_entry[len(tx_entry)-1]] + bff
            bff = sum_intervals(bff)
            tx_entry = tx_entry + bff[1:]
        else:
            bff[0] = 0
            bff = sum_intervals(bff)
            tx_entry = bff
        
        first_run = False
                
        is_run, indexTransiente = process_queue(tx_service, tx_entry, is_run)
        tx_service = tx_service[indexTransiente:]
        tx_entry = tx_entry[indexTransiente:]
        
    fim = timeit.default_timer()

    return wait,(fim-inicio)

In [9]:
start_cicle(1000, 7, 10)

KeyboardInterrupt: 